# Part 1

### 0. Start by retrieving input

In [1]:
with open('advent-5-0-rules.txt') as f:
    rules = f.read().splitlines()

with open('advent-5-0-pages.txt') as f:
    pages = f.read().splitlines()

len(pages), pages[0], len(rules), rules[0]

(197, '27,51,67,91,42,86,58,66,39,72,96', 1176, '96|15')

In [2]:
# Same with example
with open('advent-5-0-rules-ex.txt') as f:
    rules_ex = f.read().splitlines()

with open('advent-5-0-pages-ex.txt') as f:
    pages_ex = f.read().splitlines()

len(pages_ex), pages_ex[0], len(rules_ex), rules_ex[0], pages_ex[0].split(',')

(6, '75,47,61,53,29', 21, '47|53', ['75', '47', '61', '53', '29'])

In [3]:
# (1) first convert sequence of sequence numbers into a list
def separate_symbol(liste):
    list_ = []
    for i in range(len(liste)):
        pages = liste[i].split(',')
        list_.append(pages)
    return list_

pages_split = separate_symbol(pages)
pages_split_ex = separate_symbol(pages_ex)
len(pages_split), len(pages_split_ex), pages_split_ex

(197,
 6,
 [['75', '47', '61', '53', '29'],
  ['97', '61', '53', '29', '13'],
  ['75', '29', '13'],
  ['75', '97', '47', '61', '53'],
  ['61', '13', '29'],
  ['97', '13', '75', '29', '47']])

In [4]:
# (2) convert rules in lists
def separate_rules(liste):
    list_ = []
    for i in range(len(liste)):
        rule = liste[i].split('|')
        list_.append(rule)
    return list_

rules_split = separate_rules(rules)
rules_split_ex = separate_rules(rules_ex)
len(rules_split), len(rules_split_ex), rules_split_ex, '47' in rules_split_ex  # need to define a new function to get all numbers with a rule (cf below)

(1176,
 21,
 [['47', '53'],
  ['97', '13'],
  ['97', '61'],
  ['97', '47'],
  ['75', '29'],
  ['61', '13'],
  ['75', '53'],
  ['29', '13'],
  ['97', '29'],
  ['53', '29'],
  ['61', '53'],
  ['97', '53'],
  ['61', '29'],
  ['47', '13'],
  ['75', '47'],
  ['97', '75'],
  ['47', '61'],
  ['75', '61'],
  ['47', '29'],
  ['75', '13'],
  ['53', '13']],
 False)

In [5]:
def all_nb_rules(listes):
    list_ = []
    for liste in listes:
        list_.extend([liste[0], liste[1]]) # append a list of several elements
    return set(list_)

nb_rules_ex = all_nb_rules(rules_split_ex)
nb_rules = all_nb_rules(rules_split)
print(f"nb de règles vs nb de numéros impliqués = {len(rules_split_ex)} VS {len(nb_rules_ex)} (exemple), {len(rules_split)} VS {len(nb_rules)} (input)")
nb_rules_ex

nb de règles vs nb de numéros impliqués = 21 VS 7 (exemple), 1176 VS 49 (input)


{'13', '29', '47', '53', '61', '75', '97'}

### 1. We only keep pages that are in the right order (i.e do not break any rule)

In [6]:
def rules_selected(rules_, nb_str): # select only the rules in rules_ involving nb_str
    list_ = []
    for rule in rules_:
        if nb_str in rule:
            list_.append(rule)
    return list_

rules_selected(rules_split_ex, '13')

[['97', '13'],
 ['61', '13'],
 ['29', '13'],
 ['47', '13'],
 ['75', '13'],
 ['53', '13']]

In [7]:
def check_pages(pages_, rules_):
    list_ = []
    for sequence in pages_: # on parcourt chaque séquence de page
        ordered = True
        # print("============NEXT PAGE:===============")
        # print(sequence)
        for i in range(len(sequence)): # on parcourt chaque sequence à imprimer
            # print("*** New sequence ***")
            # print(f"i = {i} - {sequence[i]} - Règle? {sequence[i] in all_nb_rules(rules_)}")
            if sequence[i] in all_nb_rules(rules_): # il existe une règle pour cette page
                r = rules_selected(rules_, sequence[i]) # on récupère les règles concernées
                # print(r)
                for rule in r: # on les parcourt une à une
                    if ordered == True:
                        # print(rule)
                        if sequence[i] == rule[0]: # la sequence doit précéder
                            # print(f"{sequence[i]} doit précéder {rule[1]}")
                            if rule[1] in sequence: # l'autre nb apparait dans la séquence (else continue to next rule)
                                # print(f"l'autre nb a été trouvé et va être comparé")
                                if rule[1] in sequence[:i]: # l'autre nb apparait pourtant AVANT
                                    # print(f"PAS LE CAS")
                                    ordered = False
                                    break # exit FOR loop (rules)
                        else: # la sequence doit être après
                            # print(f"{sequence[i]} doit suivre {rule[0]}")
                            if rule[0] in sequence: # l'autre nb apparait dans la séquence (else continue to next rule)
                                # print(f"l'autre nb a été trouvé et va être comparé")
                                if rule[0] in sequence[i:]: # l'autre nb apparait pourtant APRES
                                    # print(f"PAS LE CAS")
                                    ordered = False
                                    break # exit FOR loop (rules)
                if i == len(sequence)-1 and ordered == True: # c'est la dernière sequence de la séquence et toutes les règles ont été checkées => OK
                    # print(f"fin de sequence + toutes les règles OK")
                    list_.append(sequence)
            elif i == len(sequence)-1: # c'est la dernière sequence sans règle de la séquence => OK
                # print(f"fin de sequence + dans aucune règle")
                list_.append(sequence)
    return list_

pages_filtered_ex = check_pages(pages_split_ex, rules_split_ex) # expected 75,47,61,53,29 and 97,61,53,29,13 and 75,29,13
pages_filtered_ex

[['75', '47', '61', '53', '29'],
 ['97', '61', '53', '29', '13'],
 ['75', '29', '13']]

In [8]:
pages_filtered = check_pages(pages_split, rules_split)
len(pages_filtered)

80

### 2. We keep just the middle page

In [9]:
import math

def middle_pages(listes):
    list_ = []
    for liste in listes:
        l = len(liste)
        list_.append(int(liste[math.ceil(l/2)-1]))
    return list_

middle_filtered_ex = middle_pages(pages_filtered_ex)
middle_filtered_ex # expected [61, 53, 29]

[61, 53, 29]

In [10]:
middle_filtered = middle_pages(pages_filtered)
len(middle_filtered)

80

### 3. We sum the middle pages

In [11]:
sum(middle_filtered_ex) # expected 143

143

In [12]:
sum(middle_filtered)

4569

# Part 2 (not completed)

In [13]:
# Let's filter the unordered sequences
unordered_ex = [x for x in pages_split_ex if x not in pages_filtered_ex]

print(f"{len(unordered_ex)} + {len(middle_filtered_ex)} = {len(pages_split_ex)} : {len(unordered_ex)+len(middle_filtered_ex) == len(pages_split_ex)}")


3 + 3 = 6 : True


In [14]:
unordered = [x for x in pages_split if x not in pages_filtered]

print(f"{len(unordered)} + {len(middle_filtered)} = {len(pages_split)} : {len(unordered)+len(middle_filtered) == len(pages_split)}")

117 + 80 = 197 : True


In [22]:
# Now let's order them
def order_unordered(pages_, rules_):
    list_ = []
    for sequence in pages_: # on parcourt chaque séquence de page
        for i in range(len(sequence)): # on parcourt chaque sequence à imprimer
            if sequence[i] in all_nb_rules(rules_): # il existe une règle pour cette page
                r = rules_selected(rules_, sequence[i]) # on récupère les règles concernées
                
                for rule in r: # on les parcourt une à une
                        if sequence[i] == rule[0]: # la sequence doit précéder
                            # print(f"{sequence[i]} doit précéder {rule[1]}")
                            if rule[1] in sequence: # l'autre nb apparait dans la séquence (else continue to next rule)
                                # print(f"l'autre nb a été trouvé et va être comparé")
                                if rule[1] in sequence[:i]: # l'autre nb apparait pourtant AVANT
                                    # print(f"PAS LE CAS")
                                    index_ = sequence.index(rule[1])
                                    value_ = sequence[i]
                                    sequence[i] = sequence[index_] # on intervertit
                                    sequence[index_] = value_
                        else: # la sequence doit être après
                            # print(f"{sequence[i]} doit suivre {rule[0]}")
                            if rule[0] in sequence: # l'autre nb apparait dans la séquence (else continue to next rule)
                                # print(f"l'autre nb a été trouvé et va être comparé")
                                if rule[0] in sequence[i:]: # l'autre nb apparait pourtant APRES
                                    # print(f"PAS LE CAS")
                                    index_ = sequence.index(rule[0])
                                    value_ = sequence[i]
                                    sequence[i] = sequence[index_] # on intervertit
                                    sequence[index_] = value_
                if i == len(sequence)-1: 
                    # print(f"fin de sequence + toutes les règles OK")
                    list_.append(sequence)
    return list_

pages_ordered_ex = order_unordered(unordered_ex, rules_split_ex) # 97,75,47,61,53 and 61,29,13 and 97,75,47,29,13
pages_ordered_ex

[['97', '75', '47', '61', '53'],
 ['61', '29', '13'],
 ['97', '75', '47', '29', '13']]

In [23]:
pages_ordered = order_unordered(unordered, rules_split) 
len(pages_ordered)

117

In [25]:
# Let's check if the sequences are now all ordered
pages_filtered_2 = check_pages(pages_ordered, rules_split)
len(pages_filtered_2) # expected 117

11

In [30]:
# Then let's clean the remaining unordered 
''' Infinite loop 
pages_ordered_n = pages_ordered
pages_filtered_n = check_pages(pages_ordered_n, rules_split)
n = 2
while len(pages_filtered_n) <= len(unordered): # 117
    unordered_n = [x for x in pages_ordered if x not in pages_filtered_n]
    print(len(unordered_n))
    pages_ordered_n = order_unordered(unordered_n, rules_split)
    pages_filtered_n = check_pages(pages_ordered_n, rules_split)
    n += 1
n, pages_ordered_n
'''

84
87
58
69
53
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67
52
67


KeyboardInterrupt: 

### Identify middle page & sum

In [17]:
middle_ordered_ex = middle_pages(pages_ordered_ex)
sum(middle_ordered_ex), middle_ordered_ex # expected [47, 29, 47] -> 123

(123, [47, 29, 47])

In [18]:
middle_ordered = middle_pages(pages_ordered)
sum(middle_ordered), len(middle_ordered) # ➡️ submit this answer for PART 2

(6836, 117)